In [ ]:
import langchain
from langchain.schema import Document
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import langchain_community

In [2]:
import os
import json
from datetime import datetime
import openai

In [5]:
def get_delivery_date(order_id: str) -> datetime:
    return datetime.today().strftime('%Y-%m-%d')

In [6]:
# LLM에서 사용할 함수 tool 생성
function_tool = {
    "type" : "function", # 도구의 타입을 function으로 설정 -> 함수 호출 기능 제공 명시
    "function" : {
        "name" : "get_delivery_date", # 함수 이름 정의
        "description" : "고객의 주문에 대한 배송 날짜를 확인합니다. 예를 들어, 고객이 '내 패키지가 어디에 있나요?'라고 물을 때 이 함수를 호출하세요.",
        "parameters" : {
            "type" : "object",
            "properties":{
                "order_id" : {
                "type" : "string",
                "description" : "고객의 주문 ID"
            }
         },
            "required" : ["order_id"],
            "additionalProperties" : False
        }

    }
}

tools = [function_tool]

In [7]:
messages = []
messages.append({"role": "system", "content": "당신은 도움이 되는 고객 지원 어시스턴트입니다. 제공된 도구를 사용하여 사용자를 지원하세요."})
messages.append({"role": "user", "content": "안녕하세요, 제 주문의 배송 날짜를 알려주실 수 있나요?"})
messages.append({"role": "assistant", "content": "안녕하세요! 제가 도와드릴 수 있습니다. 주문 ID를 알려주시겠어요?"})
messages.append({"role": "user", "content": "쓸데 없는 소리"})

In [ ]:
# 함수 매개변수 추출
tool_call = response.choices[0].tool_calls

if tool_call:
    print(tool_call[0])
else:
    print("No") #실제론 필요 없음

In [ ]:
arguments = json.loads(tool_call[0].function.arguments)
arguments

In [ ]:
get_delivery_date(**arguments)

In [ ]:
# 데이터베이스에 직접 접근하는 함수 or Spring API를 호출하는 함수를 정의
def search_hanger(height : str, width : str, depth : str) -> dict:
    # 실제 쿼리 접근 코드 또는 api 호출 코드
    # api 접근
    # import requests
    # response = requests.get(f"https://YOUR_API_URL/height={height}&width={width}&depth={depth}")
    # db
    # connection_str = "mysql:~~~~/db"
    print("DB 쿼리 및 api 활용 수행")
    search_result = {
    "hangerRacks": [
      {
        "id": "HR001",
        "description": "5-tier hanger rack",
        "dimensions": {
          "width": 800,
          "depth": 600,
          "height": 2100
        },
        "tiers": 5,
        "quantity": 2,
        "material": "Steel",
        "color": "Black",
        "loadCapacityPerTier": 50,
        "totalLoadCapacity": 250,
        "features": [
          "Adjustable shelves",
          "Anti-rust coating",
          "Non-slip feet"
        ]
      },
      {
        "id": "HR002",
        "description": "3-tier double hanger rack",
        "dimensions": {
          "width": 800,
          "depth": 600,
          "height": 2100
        },
        "tiers": 3,
        "quantity": 2,
        "material": "Steel",
        "color": "White",
        "loadCapacityPerTier": 40,
        "totalLoadCapacity": 240,
        "features": [
          "Double hanging rods",
          "Wheels for mobility",
          "Adjustable height"
        ]
      }
    ]
  }
    return search_result

search_hanger("a", "b", "c")

In [4]:
# llm이 search_hanger 함수를 알아낼 수 있게 도구를 정의
search_tool = {
    "type": "function",
    "function":{
        "name": "search_hanger",
        "description": "고객이 찾고 있는 행거에 대한 검색 결과를 반환합니다. 고객이 행거에 대한 높이(height), 너비(width), 깊이(depth)를 정확히 입력한 경우에 이 함수를 호출하세요.",

        "parameters": {
            "type": "object",
            "properties": {
                "height": {
                    "type": "string",
                    "description": "행거의 높이"
                },
                "width": {
                    "type": "string",
                    "description": "행거의 너비(길이)"
                },
                "depth": {
                    "type": "string",
                    "description": "행거의 깊이"
                }
            },
            "required": ["height", "width", "depth"],
            "additionalProperties": False
        }
    }
}
tools = [search_tool]

In [5]:
system_prompt = """

당신은 검색된 문서부터 질문의 답변을 작성하는 언어 모델입니다. 도구를 이용해 사용자를 지원합니다.

### 지시사항
당신은 사용자로부터 선반 랙의 높이, 너비(길이), 깊이를 확정받아서 검색을 수행하는 챗봇입니다.

1. 사용자가 행거의 높이(세로), 길이(가로), 깊이 이 세 가지 값을 모두 확정할 때까지 사용자에게 반문하세요. 이는 가장 중요합니다. 모든 것이 결정될 때까지 반문하십시오.
2. 단수 추가를 요청하는 경우 1세트 설치 시 2~3cm 여유공간이 필요하다고 안내하세요.
3. 모든 것이 확정되면 도구를 사용합니다.

예시)

User: 길이 3000, 높이 2100의 선반 랙을 사려고합니다.
Assistant: 네, 설치하고자 하는 선반 랙의 길이가 3000, 높이 2100이 맞으실까요? 깊이에 대한 정보도 알려주시면 선반을 검색해드리겠습니다.
User: 실제 공간 총 높이는 2300인데 조금 비워두려고 2100으로 하려합니다. 그리고 향후에 단수 추가되는 부분도 추가로 구성이 가능할까요?
Assistant: 1세트 설치 시 2~3cm 여유공간이 필요하신 점 참고 부탁드립니다.깊이는 500 정도로 괜찮으실까요?
User: 깊이는 700이 좋겠습니다.
Assistant: 네, 선반을 추천해드릴게요.
검색 결과:
"""

In [6]:
user_prompt = ""

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "assistant", "content": "안녕하세요! 선반 랙을 찾고 계신가요? 설치하고자 하는 선반 랙의 높이, 너비(길이), 깊이를 알려주시면 검색을 도와드리겠습니다. 세 가지 정보를 모두 제공해 주시면 감사하겠습니다."},
    {"role": "user", "content": "아 선반 랙은 분홍색으로 찾고 있는데 높이는 1500 정도가 좋겠어요."},
    {"role": "assistant", "content": "네, 선반 랙의 높이가 1500으로 설정하셨군요. 이제 너비(길이)와 깊이에 대한 정보를 알려주시면 검색을 도와드리겠습니다."},
    {"role": "user", "content": "음 고민해볼게요. 혹시 향후에 단수 추가되는 부분도 추가로 구성이 가능할까요?"},
    {"role": "assistant", "content": "1세트 설치 시 2~3cm 여유공간이 필요하신 점 참고 부탁드립니다. 단수 추가를 고려하신다면 이 점을 염두에 두시면 좋겠습니다. 너비(길이)와 깊이에 대한 정보도 알려주시면 검색을 도와드리겠습니다."},
    {"role": "user", "content": "아 정정할게요. 높이는 1200, 길이는 2000으로요."},
    {"role": "assistant", "content": "네, 선반 랙의 높이가 1200, 길이가 2000으로 설정하셨군요. 이제 깊이에 대한 정보를 알려주시면 검색을 도와드리겠습니다."},
    {"role": "user", "content": "깊이 추천 좀 해주세요."},
    {"role": "assistant", "content": "일반적으로 선반 랙의 깊이는 사용 용도에 따라 다르지만, 보통 400mm에서 600mm 사이가 많이 사용됩니다. 만약 더 많은 수납 공간이 필요하시다면 700mm 이상의 깊이도 고려해보실 수 있습니다. 어떤 깊이가 좋으신가요?"},
    {"role": "user", "content": "사무실에서 쓸 선반이에요"},
    {"role": "assistant", "content": "사무실에서 사용하실 경우, 일반적으로 400mm에서 600mm 깊이의 선반이 적당합니다. 이 범위 내에서 선택하시면 좋을 것 같습니다. 어떤 깊이로 하시겠어요?"},
    {"role": "user", "content": "500 정도가 좋아보여요"},
  ],
  temperature=0,
  tools=tools
)

print(response.choices[0].message.content)

if response.choices[0].finish_reason == "tool_calls":
  arguments = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
  search_result = search_hanger(**arguments)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "assistant", "content": "안녕하세요! 선반 랙을 찾고 계신가요? 설치하고자 하는 선반 랙의 높이, 너비(길이), 깊이를 알려주시면 검색을 도와드리겠습니다. 세 가지 정보를 모두 제공해 주시면 감사하겠습니다."},
    {"role": "user", "content": "아 선반 랙은 분홍색으로 찾고 있는데 높이는 1500 정도가 좋겠어요."},
    {"role": "assistant", "content": "네, 선반 랙의 높이가 1500으로 설정하셨군요. 이제 너비(길이)와 깊이에 대한 정보를 알려주시면 검색을 도와드리겠습니다."},
    {"role": "user", "content": "음 고민해볼게요. 혹시 향후에 단수 추가되는 부분도 추가로 구성이 가능할까요?"},
    {"role": "assistant", "content": "1세트 설치 시 2~3cm 여유공간이 필요하신 점 참고 부탁드립니다. 단수 추가를 고려하신다면 이 점을 염두에 두시면 좋겠습니다. 너비(길이)와 깊이에 대한 정보도 알려주시면 검색을 도와드리겠습니다."},
    {"role": "user", "content": "아 정정할게요. 높이는 1200, 길이는 2000으로요."},
    {"role": "assistant", "content": "네, 선반 랙의 높이가 1200, 길이가 2000으로 설정하셨군요. 이제 깊이에 대한 정보를 알려주시면 검색을 도와드리겠습니다."},
    {"role": "user", "content": "깊이 추천 좀 해주세요."},
    {"role": "assistant", "content": "일반적으로 선반 랙의 깊이는 사용 용도에 따라 다르지만, 보통 400mm에서 600mm 사이가 많이 사용됩니다. 만약 더 많은 수납 공간이 필요하시다면 700mm 이상의 깊이도 고려해보실 수 있습니다. 어떤 깊이가 좋으신가요?"},
    {"role": "user", "content": "사무실에서 쓸 선반이에요"},
    {"role": "assistant", "content": "사무실에서 사용하실 경우, 일반적으로 400mm에서 600mm 깊이의 선반이 적당합니다. 이 범위 내에서 선택하시면 좋을 것 같습니다. 어떤 깊이로 하시겠어요?"},
    {"role": "user", "content": "500 정도가 좋아보여요"},
    {"role": "assistant", "content": "좋습니다! 선반 랙의 높이는 1200, 길이는 2000, 깊이는 500으로 설정하셨습니다. 이제 이 정보를 바탕으로 선반 랙을 검색해드리겠습니다."},

    # 아래 내용은 굳이 사용자에게 노출할 필요 없이(UI 없이) 사용하면 됩니다.(프로그래밍 적인 처리 필요)
    {"role": "user", "content": f"검색 결과: {search_result} 자 이제 검색 결과를 바탕으로 사용자에게 전형적인 LLM 답변을 작성하세요."},
  ],
  temperature=0,
  tools=tools
)

print(response.choices[0].message.content)